In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from scipy import stats
import biolearn
from biolearn.data_library import DataLibrary

def df2dt(df,man):
    dt=df.loc[df.index.intersection(man.index)]
    dt=dt.dropna(axis=1, thresh=len(dt) * 0.8).copy()
    dt=dt.dropna(thresh=len(dt.columns) * 0.8).copy()
    return dt

def get_cor(dt):
    regs=[]
    x=np.array(dt)
    y=dt['age']
    for i in range(len(x[0])):
        mask = ~np.isnan(x[:,i]) & ~np.isnan(y)
        regs.append(stats.linregress(y[mask],x[:,i][mask]))
    cor=pd.DataFrame(np.array(regs)[:,2:4],index=dt.columns,columns=['cor','p']).sort_values('cor',ascending=False)
    return cor
data='../data/'
cache=biolearn.cache.LocalFolderCache(data+'cache',1000)

n=6

In [2]:
%%time
man=pd.read_csv(data+'clean/man_e1.csv',index_col=0)
man.shape

CPU times: user 2.94 s, sys: 377 ms, total: 3.31 s
Wall time: 3.32 s


(846233, 13)

In [62]:
%%time
meta=pd.read_csv(data+'clean/cohorts/meta/mesa_dna.csv',index_col=0)
df=pd.read_pickle(data+'pkls/dna/mesa.pkl')
df=df[[c for c in meta.index]]
dt=df2dt(df,man)

CPU times: user 29.7 s, sys: 1min 2s, total: 1min 32s
Wall time: 2min


In [63]:
%%time
dt=meta[['visit','age','sid']].merge(dt.T,left_index=True,right_index=True)
dt[['age','visit','sid','cg16867657','cg08097417','cg06639320','cg22213242','cg09837977','cg02756106','cg26768584']].to_csv(data+'results/top_cgs_mesa.csv') #for Fig2
d1=dt[dt['visit']==1].drop(['sid','visit'],axis=1).copy()
d2=dt[dt['visit']==2].drop(['sid','visit'],axis=1).copy()

CPU times: user 17.1 s, sys: 22.3 s, total: 39.4 s
Wall time: 39.6 s


In [53]:
%%time
cor1=get_cor(d1)
cor2=get_cor(d2)
cor1['cohort']='mesa1'
cor2['cohort']='mesa2'
cor=pd.concat([cor1,cor2]).drop('age')
cor.to_csv(data+'results/dna_corr/mesa.csv')

CPU times: user 13min 7s, sys: 29.2 s, total: 13min 37s
Wall time: 13min 39s


In [64]:
%%time
cohort='ppmi'
df=pd.read_pickle(data+'pkls/dna/ppmi.pkl')
dt=df2dt(df,man)
age=pd.read_csv(data+'clean/cohorts/meta/ppmi_dna.csv',index_col=0)['age']
dt=dt.T.join(age,how='inner')
dt[['age','cg16867657','cg08097417','cg06639320','cg22213242','cg09837977','cg02756106','cg26768584']].to_csv(data+'results/top_cgs_ppmi.csv')  #for Fig2
cor=get_cor(dt).drop('age')
cor['cohort']=cohort
cor.to_csv(data+'results/dna_corr/ppmi.csv')

CPU times: user 5min 51s, sys: 29.9 s, total: 6min 21s
Wall time: 6min 30s


In [61]:
%%time
cohort='grady'
df=pd.read_pickle(data+'pkls/dna/grady.pkl')
dt=df2dt(df,man)
age=pd.read_csv(data+'clean/cohorts/meta/grady.csv',index_col=0)['age']
dt=dt.T.join(age,how='inner')
cor=get_cor(dt).drop('age')
cor['cohort']=cohort
cor.to_csv(data+'results/dna_corr/grady.csv')

CPU times: user 7min 17s, sys: 47.2 s, total: 8min 4s
Wall time: 8min 6s


In [8]:
%%time
cohort='geno'
df=pd.read_pickle(data+'pkls/dna/geno.pkl')
dt=df2dt(df,man)
geo='GSE157131'
meta = DataLibrary(cache=cache).get(geo).load().metadata
meta['sex']=meta['sex']-1
dt=meta[['age']].join(df.T)
cor=get_cor(dt).drop('age')
cor['cohort']=cohort
cor.to_csv(data+'results/dna_corr/geno.csv')

CPU times: user 3min 37s, sys: 24.5 s, total: 4min 2s
Wall time: 4min 9s


In [9]:
%%time
cohort='ra'
df=pd.read_pickle(data+'pkls/dna/ra.pkl')
dt=df2dt(df,man)
geo='GSE42861'
meta = DataLibrary(cache=cache).get(geo).load().metadata
meta['sex']=meta['sex']-1
dt=meta[['age']].join(df.T)
cor=get_cor(dt).drop('age')
cor['cohort']=cohort
cor.to_csv(data+'results/dna_corr/ra.csv')

CPU times: user 3min 52s, sys: 19.4 s, total: 4min 11s
Wall time: 4min 18s


In [10]:
# MGB is EPIC2

In [11]:
%%time
man2=pd.read_csv(data+'clean/man_e2.csv',index_col=0)
man2.shape

CPU times: user 2.95 s, sys: 300 ms, total: 3.25 s
Wall time: 3.25 s


(905622, 14)

In [12]:
%%time
cohort='mgb'
df=pd.read_pickle(data+'pkls/dna/mgb.pkl').T
### df mgb.pkl is already cleaned and filtered so no need to do dt=df2dt(df,man2)
dt=df.copy()
age=pd.read_csv(data+'clean/cohorts/meta/mgb.csv',index_col=0)['age_at_sample'].rename('age')
dt=dt.join(age,how='inner')
cor=get_cor(dt).drop('age')
cor['cohort']=cohort
cor.to_csv(data+'results/dna_corr/mgb.csv')
cor.shape

CPU times: user 5min 46s, sys: 10.7 s, total: 5min 56s
Wall time: 5min 57s


(825095, 3)

In [ ]:
%%time
ppmi=pd.read_csv(data+'results/dna_corr/ppmi.csv',index_col=0)
grady=pd.read_csv(data+'results/dna_corr/grady.csv',index_col=0)
geno=pd.read_csv(data+'results/dna_corr/geno.csv',index_col=0)
ra=pd.read_csv(data+'results/dna_corr/ra.csv',index_col=0)
mesa=pd.read_csv(data+'results/dna_corr/mesa.csv',index_col=0)
mgb=pd.read_csv(data+'results/dna_corr/mgb.csv',index_col=0);mgb['cohort']='mgb'
dna=pd.concat([mesa,ppmi,ra,grady,geno,mgb])
# CpGs from Kaplan paper
clock=pd.read_excel(data+'source/kaplan.xlsx',index_col=0).iloc[:-1].sort_values('Spearman correlation',ascending=False)
clock.loc[clock['Spearman correlation']>0,'dir']='gain'
clock.loc[clock['Spearman correlation']<0,'dir']='loss'
gain=clock[clock['dir']=='gain']
loss=clock[clock['dir']=='loss']
gain10=clock[clock['dir']=='gain'].head(10)
loss10=clock[clock['dir']=='loss'].tail(10)
len(gain),len(loss),len(gain10),len(loss10)
dna.loc[dna.index.isin(gain.index),'proposed']='up'
dna.loc[dna.index.isin(loss.index),'proposed']='down'
dna.loc[dna.index.isin(gain10.index),'top10']='up'
dna.loc[dna.index.isin(loss10.index),'top10']='down'
cohorts={'mesa1':'MESA1','mesa2':'MESA2','ppmi':'PPMI','ra':'RA','grady':'Grady','geno':'GENOA','mgb':'MGB'}
dna['cohort']=dna['cohort'].replace(cohorts)
top=dna[dna['top10'].isin(['up','down'])].copy()
top['cg']=top.index
dna.to_csv(data+'results/dna_cor.csv')
fig1f=top[['cor','cohort','cg','top10']]
fig1h=dna[dna['cohort']!='MGB'][['cor','cohort','proposed']]
fidg3b=dna[['cor','cohort','proposed']]
fig1f.to_csv(data+'figs/1f.csv')
fig1h.to_csv(data+'figs/1h.csv')
fig1h.to_csv(data+'figs/3b.csv')